# IoU pixel by pixel

In [27]:
import numpy as np
import keras
from keras.layers import *
import keras.backend as K
from keras.models import Model

In [70]:
def compute_IoU(y_true, y_pred):
    # convert to numpy array for convenience and flatten
    pred = K.eval(y_pred)
    pred_f = pred.flatten()
    truth = K.eval(y_true)
    truth_f = truth.flatten()
    
    out_shape = pred.shape
        
    IoU = np.zeros(out_shape)
    
    lbls_in_pred = np.unique(pred_f)
    for pr_class in lbls_in_pred:
        # create mask of units that predicted pr_class
        mask_pr = (pred_f == pr_class)
        mask_tr = (truth_f == pr_class)
        
        # get intersection and union values for pr_class
        inter = (mask_pr == mask_tr)
        union = np.sum(mask_pr + mask_tr ^ inter)
        
        iou = (sum(inter) / union)
        # use value only on units that did the prediction
        iou_arr = mask_pr * iou
        
        # reshape and put in dictionary
        IoU += iou_arr.reshape(out_shape)
    
    # return the complementary of IoU to punish more low IoU 
    return K.variable(1 - IoU)
    

In [76]:
# pred = np.random.randint(0, 6, (300, 300))
# truth = np.random.randint(0, 6, (300, 300))

shp = (30, 30)

pred = np.random.randint(0, 6, shp)

truth = np.ones(shp)

pred = K.variable(pred)
truth = K.variable(truth)

c = compute_IoU(truth, pred)


In [86]:
l = keras.losses.categorical_crossentropy(truth, pred)
print(K.eval(l).shape)
# print()
print(K.eval(l + l * c).shape)

(30,)
(30, 30)
